<a href="https://colab.research.google.com/github/yatta03/112-2PL/blob/main/project/med_panadol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# set up chromedriver for selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

!pip install selenium
!pip install pygsheets

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:7 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,085 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,858 kB]
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://developer.download.nvidi

In [2]:
# import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
import json
import pandas as pd
import pygsheets
import re

In [3]:
class Crawler:
  def __init__(self):
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument("--disable-gpu")  # Required for Windows
    chrome_options.add_argument('--no-sandbox')
    chrome_options.headless = True
    self.driver = webdriver.Chrome(options=chrome_options)

  # wait until the button to be clickable, then click
  def waitClickBtn(self, byType, byName):
    try:
      element = WebDriverWait(self.driver, 5).until(EC.element_to_be_clickable((byType, byName)), "not found")
      element.click()
    except Exception as e:
      print(e)

  #  wait the locator or waittime to load the content
  def waitContent(self, waitTime, locator = None):
    if locator == None:
      self.driver.implicitly_wait(waitTime)
    else:
      WebDriverWait(self.driver, waitTime).until(EC.presence_of_element_located(locator), "not found")

  # wait until locator presents and return parsed html of given url
  def getSoup(self, url, locator = None):
    self.driver.get(url)
    if(locator):
      try:
          WebDriverWait(self.driver, 10).until(EC.presence_of_element_located(locator), "not found")
      except:
          print("locator not found")
    else:
      self.driver.implicitly_wait(5)
    htmlContent = self.driver.page_source

    soup = bs(htmlContent, 'html.parser')
    return soup

  # quit the driver
  def quit(self):
    self.driver.quit()

In [5]:
panadol_url = "https://www.panadol-cold-flu.com.tw/productlist.html"

In [9]:
# get all medicine page links
def get_med_links():
  links = []
  crawler = Crawler()

  crawler.getSoup(panadol_url)
  meditems = crawler.driver.find_elements(By.CLASS_NAME, "pblock")
  for med in meditems:
    l = med.find_element(By.TAG_NAME, "a").get_attribute("href")
    links.append(l)

  crawler.quit()
  return links

In [25]:
# get medicine info from link
def get_med(link):
  crawler = Crawler()
  c = crawler.getSoup(link)

  med = {}
  img_path = crawler.driver.find_element(By.CLASS_NAME, "productImgBig").get_attribute("src")
  name = crawler.driver.find_element(By.XPATH, "//h2[@role='heading']").text

  symptom = crawler.driver.find_elements(By.XPATH, "//ul[@class='ftable']/li")
  symptom = [s.text for s in symptom]

  ingredients = crawler.driver.find_elements(By.XPATH, "//div[@class='specTable']/table/tbody/tr/td[1]")
  ingredients = [i.text for i in ingredients]

  med["name"] = name
  med["img_path"] = img_path
  med["app_txt"] = ""
  med["symptom"] = symptom
  med["ingredient"] = ingredients
  med["link"] = link

  crawler.quit()
  return med

In [27]:
medicines = []
links = []

links = get_med_links()
for l in links:
  medicines.append(get_med(l))

panadol_med = pd.DataFrame(medicines)
panadol_med.head()

,name,img_path,app_txt,symptom,ingredient,link
0,伏冒熱飲,https://www.panadol-cold-flu.com.tw/images/pth...,,"[咽喉痛, 發燒, 頭痛, 肌肉酸痛, 鼻塞, 關節痛]","[Acetaminophen, Phenylephrine Hydrochloride, A...",https://www.panadol-cold-flu.com.tw/product07....
1,伏冒綜合鼻炎錠,https://www.panadol-cold-flu.com.tw/images/pth...,,"[咽喉痛, 發燒, 頭痛, 關節痛, 肌肉酸痛, 鼻塞, 流鼻水, 打噴嚏]","[Acetaminophen, Pseudoephedrine Hydrochloride,...",https://www.panadol-cold-flu.com.tw/product04....
2,伏冒膜衣錠,https://www.panadol-cold-flu.com.tw/images/pth...,,"[咽喉痛, 發燒, 頭痛, 關節痛, 肌肉酸痛, 鼻塞]","[Acetaminophen, Phenylephrine Hydrochloride, C...",https://www.panadol-cold-flu.com.tw/product02....
3,伏冒熱飲加強配方,https://www.panadol-cold-flu.com.tw/images/pth...,,"[咳嗽, 咽喉痛, 畏寒, 發燒, 頭痛, 關節痛, 肌肉酸痛, 鼻塞]","[Acetaminophen, Phenylephrine Hydrochloride, D...",https://www.panadol-cold-flu.com.tw/product06....
4,伏冒加強錠,https://www.panadol-cold-flu.com.tw/images/pth...,,"[咳嗽, 咽喉痛, 畏寒, 發燒, 頭痛, 肌肉酸痛, 鼻塞]","[Acetaminophen, Phenylephrine HCl, Noscapine, ...",https://www.panadol-cold-flu.com.tw/product01....


In [28]:
gc = pygsheets.authorize(service_file='./pl-421203-a74b8641e6bc.json')
sheetUrl = "https://docs.google.com/spreadsheets/d/16_34--12pFGZ3alnL_mAuCEE5ne0aaxZ0sd8qlJzyOo/edit?usp=sharing"
sh = gc.open_by_url(sheetUrl)

In [29]:
def dfToSheet(df, sh, title):
  try:
    ws = sh.worksheet_by_title(title)
    ws.set_dataframe(df, start=(0,0), nan='')
  except Exception as e:
    print(e)
def clearSheet(sh, title):
  try:
    ws = sh.worksheet_by_title(title)
    ws.clear()
  except Exception as e:
    print(e)

In [31]:
clearSheet(sh, "panadol")
dfToSheet(panadol_med, sh, "panadol")